# Music Store Analysis with PostgreSQL

## PARTIE 1 : Requêtes SQL

In [32]:
#connexion à la base de de données

import psycopg2
import pandas as pd

# Configuration de la connexion
connection = psycopg2.connect(
    host="localhost",
    database="music_store_database",
    user="postgres",
    password="root",
    port="5432"  # Par défaut, le port PostgreSQL est 5432
)

# Fonction pour exécuter une requête SQL et retourner les résultats sous forme de DataFrame
def execute_query(query):
    with connection.cursor() as cursor:
        cursor.execute(query)
        # Récupérer les noms des colonnes
        colnames = [desc[0] for desc in cursor.description]
        # Récupérer les données
        rows = cursor.fetchall()
        # Créer un DataFrame pour afficher les données
        return pd.DataFrame(rows, columns=colnames)



## Question Set 1 - Easy
### Who is the senior most employee based on job title?

In [33]:
query1="SELECT title, first_name, last_name FROM employee ORDER BY title DESC LIMIT 1;"
Q1= execute_query(query1)
Q1

,title,first_name,last_name
0,Senior General Manager,Mohan ...,Madan ...


### Which countries have the most Invoices?

In [34]:
# Fermer la connexion en cas de transaction échouée
connection.rollback()

query2="SELECT COUNT(*), billing_country FROM invoice GROUP BY billing_country ORDER BY COUNT(*) DESC;"
Q2 = execute_query(query2)
Q2

,count,billing_country
0,131,USA
1,76,Canada
2,61,Brazil
3,50,France
4,41,Germany
5,30,Czech Republic
6,29,Portugal
7,28,United Kingdom
8,21,India
9,13,Chile


### What are top 3 values of total invoice?

In [35]:
query3="SELECT total FROM invoice ORDER BY total DESC LIMIT 3;"
Q3=execute_query(query3)
Q3

,total
0,23.76
1,19.80
2,19.80


### Which city has the best customers? We would like to throw a promotional Music Festival in the city we made the most money. Write a query that returns one city that has the highest sum of invoice totals. Return both the city name & sum of all invoice totals

In [36]:
query4="SELECT billing_city,SUM(total) FROM invoice GROUP BY billing_city ORDER BY SUM(total) DESC LIMIT 1;"
Q4=execute_query(query4)
Q4

,billing_city,sum
0,Prague,273.24


### Who is the best customer? The customer who has spent the most money will bedeclared the best customer. Write a query t hat returns the person who has spent the most money

In [37]:
query5="SELECT customer.customer_id, first_name, last_name, SUM(total) AS total_spending FROM customer JOIN invoice ON customer.customer_id = invoice.customer_id GROUP BY customer.customer_id ORDER BY total_spending DESC LIMIT 1;"
Q5=execute_query(query5)
Q5

,customer_id,first_name,last_name,total_spending
0,5,František ...,Wichterlová ...,144.54


## Question Set 2 – Moderate
### Write query to return the email, first name, last name, & Genre of all Rock Music listeners. Return your list ordered alphabetically by email starting with A

In [38]:
query6="SELECT customer.email, customer.first_name, customer.last_name, genre.name AS genre FROM customer, genre WHERE genre.name LIKE '%Rock%' and customer.email LIKE 'a%' ORDER BY email ASC;"
Q6=execute_query(query6)
Q6

,email,first_name,last_name,genre
0,aaronmitchell@yahoo.ca,Aaron ...,Mitchell ...,Rock
1,aaronmitchell@yahoo.ca,Aaron ...,Mitchell ...,Rock And Roll
2,alero@uol.com.br,Alexandre ...,Rocha ...,Rock
3,alero@uol.com.br,Alexandre ...,Rocha ...,Rock And Roll
4,astrid.gruber@apple.at,Astrid ...,Gruber ...,Rock
5,astrid.gruber@apple.at,Astrid ...,Gruber ...,Rock And Roll


### Let's invite the artists who have written the most rock music in our dataset. Write a query that returns the Artist name and total track count of the top 10 rock bands

In [39]:
query7="SELECT artist.name AS artist_name, COUNT(track.track_id) AS track_count FROM artist JOIN album ON artist.artist_id = album.artist_id JOIN track ON album.album_id = track.album_id JOIN genre ON track.genre_id = genre.genre_id WHERE genre.name = 'Rock' GROUP BY artist.name ORDER BY track_count DESC LIMIT 10;" 
Q7 = execute_query(query7)
Q7

,artist_name,track_count
0,Led Zeppelin,114
1,U2,112
2,Deep Purple,92
3,Iron Maiden,81
4,Pearl Jam,54
5,Van Halen,52
6,Queen,45
7,The Rolling Stones,41
8,Creedence Clearwater Revival,40
9,Kiss,35


### Return all the track names that have a song length longer than the average song length. Return the Name and Milliseconds for each track. Order by the song length with the longest songs listed first

In [40]:
query8="SELECT track.name, track.milliseconds FROM track WHERE track.milliseconds > (SELECT AVG(milliseconds) FROM track) ORDER BY track.milliseconds DESC;" 
Q8 = execute_query(query8)
Q8

,name,milliseconds
0,Occupation / Precipice,5286953
1,Through a Looking Glass,5088838
2,"Greetings from Earth, Pt. 1",2960293
3,The Man With Nine Lives,2956998
4,"Battlestar Galactica, Pt. 2",2956081
...,...,...
489,22 Acacia Avenue,395572
490,The Unforgiven II,395520
491,The Shortest Straw,395389
492,"Concerto for Clarinet in A Major, K. 622: II. ...",394482


## Question Set 3 – Advance
### Find how much amount spent by each customer on artists? Write a query to return customer name, artist name and total spent

In [41]:
query9="""WITH best_selling_artist AS (
    SELECT 
        artist.artist_id AS artist_id, 
        artist.name AS artist_name, 
        SUM(invoice_line.unit_price * invoice_line.quantity) AS total_sales
    FROM 
        invoice_line
    JOIN 
        track ON track.track_id = invoice_line.track_id
    JOIN 
        album ON album.album_id = track.album_id
    JOIN 
        artist ON artist.artist_id = album.artist_id
    GROUP BY 
        artist.artist_id, artist.name
    ORDER BY 
        total_sales DESC
)
SELECT 
    c.first_name || ' ' || c.last_name AS customer_name, 
    bsa.artist_name, 
    SUM(il.unit_price * il.quantity) AS total_spent
FROM 
    invoice i
JOIN 
    customer c ON c.customer_id = i.customer_id
JOIN 
    invoice_line il ON il.invoice_id = i.invoice_id
JOIN 
    track t ON t.track_id = il.track_id
JOIN 
    album alb ON alb.album_id = t.album_id
JOIN 
    best_selling_artist bsa ON bsa.artist_id = alb.artist_id
GROUP BY 
    customer_name, bsa.artist_name
ORDER BY 
    total_spent DESC;
"""
Q9=execute_query(query9)
Q9

,customer_name,artist_name,total_spent
0,Hugh O'Reilly,Queen,27.72
1,Wyatt Girard,Frank Sinatra,23.76
2,Helena Holý,Red Hot Chili Peppers,19.80
3,Robert Brown,Creedence Clearwater Revival,19.80
4,François Tremblay,The Who,19.80
...,...,...,...
2184,Joakim Johansson,Faith No More,0.99
2185,Richard Cunningham,Caetano Veloso,0.99
2186,Ellie Sullivan,Audioslave,0.99
2187,Stanisław Wójcik,Deep Purple,0.99


### We want to find out the most popular music Genre for each country. We determine the most popular genre as the genre with the highest amount of purchases. Write a query that returns each country along with the top Genre. For countries where the maximum number of purchases is shared return all Genres

In [42]:
query10="""WITH genre_purchases AS (
    SELECT 
        customer.country, 
        genre.name AS genre_name, 
        COUNT(invoice_line.quantity) AS purchases
    FROM 
        invoice_line
    JOIN 
        invoice ON invoice.invoice_id = invoice_line.invoice_id
    JOIN 
        customer ON customer.customer_id = invoice.customer_id
    JOIN 
        track ON track.track_id = invoice_line.track_id
    JOIN 
        genre ON genre.genre_id = track.genre_id
    GROUP BY 
        customer.country, genre.name
),
max_purchases AS (
    SELECT 
        country, 
        MAX(purchases) AS max_purchases
    FROM 
        genre_purchases
    GROUP BY 
        country
)
SELECT 
    gp.country, 
    gp.genre_name, 
    gp.purchases
FROM 
    genre_purchases gp
JOIN 
    max_purchases mp ON gp.country = mp.country AND gp.purchases = mp.max_purchases
ORDER BY 
    gp.country, gp.genre_name;
""" 
Q10=execute_query(query10)
Q10 

,country,genre_name,purchases
0,Argentina,Alternative & Punk,17
1,Australia,Rock,34
2,Austria,Rock,40
3,Belgium,Rock,26
4,Brazil,Rock,205
5,Canada,Rock,333
6,Chile,Rock,61
7,Czech Republic,Rock,143
8,Denmark,Rock,24
9,Finland,Rock,46


### Write a query that determines the customer that has spent the most on music for each country. Write a query that returns the country along with the top customer and how much they spent. For countries where the top amount spent is shared, provide all customers who spent this amount

In [43]:
query11="""WITH Customer_with_country AS (
    SELECT 
        customer.customer_id,
        customer.first_name,
        customer.last_name,
        billing_country,
        SUM(invoice.total) AS total_spending
    FROM 
        invoice
    JOIN 
        customer ON customer.customer_id = invoice.customer_id
    GROUP BY 
        customer.customer_id, customer.first_name, customer.last_name, billing_country
),
Max_spending_per_country AS (
    SELECT 
        billing_country,
        MAX(total_spending) AS max_spending
    FROM 
        Customer_with_country
    GROUP BY 
        billing_country
)
SELECT 
    cwc.customer_id,
    cwc.first_name,
    cwc.last_name,
    cwc.billing_country,
    cwc.total_spending
FROM 
    Customer_with_country cwc
JOIN 
    Max_spending_per_country mspc ON cwc.billing_country = mspc.billing_country AND cwc.total_spending = mspc.max_spending
ORDER BY 
    cwc.billing_country, cwc.total_spending DESC;
""" 
Q11=execute_query(query11)
Q11 

,customer_id,first_name,last_name,billing_country,total_spending
0,56,Diego ...,Gutiérrez ...,Argentina,39.60
1,55,Mark ...,Taylor ...,Australia,81.18
2,7,Astrid ...,Gruber ...,Austria,69.30
3,8,Daan ...,Peeters ...,Belgium,60.39
4,1,Luís ...,Gonçalves ...,Brazil,108.90
5,3,François ...,Tremblay ...,Canada,99.99
6,57,Luis ...,Rojas ...,Chile,97.02
7,5,František ...,Wichterlová ...,Czech Republic,144.54
8,9,Kara ...,Nielsen ...,Denmark,37.62
9,44,Terhi ...,Hämäläinen ...,Finland,79.20


## Partie 2 : Visualisation avec R

In [46]:
# Installer les packages si tu ne les as pas déjà
install.packages("dplyr")
install.packages("ggplot2")
install.packages("lubridate")
install.packages("ggrepel")
install.packages("tidyr")


NameError: name 'install' is not defined